(EN)
Implement an LCEL chain with a memory that uses one of the memory classes we learned about.
The chain should take the title of a movie and reply with three emojis that represent the movie. (i.e "Top Gun" -> "🛩️👨‍✈️🔥". "The Godfather" -> "👨‍👨‍👦🔫🍝 ").
Provide examples to the chain using FewShotPromptTemplate or FewShotChatMessagePromptTemplate to make sure it always replies with three emojis.
To check that the memory is working ask the chain about two movies and then in another cell ask the chain to tell you what is the movie you asked about first.

(KR)
앞서 배운 메모리 클래스 중 하나를 사용하는 메모리로 LCEL 체인을 구현합니다.
이 체인은 영화 제목을 가져와 영화를 나타내는 세 개의 이모티콘으로 응답해야 합니다. (예: "탑건" -> "🛩️👨‍✈️🔥". "대부" -> "👨‍👨‍👦🔫🍝").
항상 세 개의 이모티콘으로 답장하도록 FewShotPromptTemplate 또는 FewShotChatMessagePromptTemplate을 사용하여 체인에 예시를 제공하세요.
메모리가 작동하는지 확인하려면 체인에 두 개의 영화에 대해 질문한 다음 다른 셀에서 체인에 먼저 질문한 영화가 무엇인지 알려달라고 요청하세요.

In [1]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate

llm = ChatOpenAI(temperature=0.1,
)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    return_messages=True,
)

examples=[ 
    {
       "question":"What do you know about Top Gun?",
       "answer":"""
       🛩️👨‍✈️🔥
       """,
    },
    {
       "question":"What do you know about The Godfather?",
       "answer":"""
       👨‍👨‍👦🔫🍝
       """,
    },
    {
       "question":"What do you know about Jaws?",
       "answer":"""
      🛥️☠️🦈
      """,
    },
    {
       "question":"What do you know about The Ring?",
       "answer":"""
      📺⚫️🧟‍♀️
      """,
    },
]

example_prompt = ChatPromptTemplate.from_messages([
    ("human", "{question}"),
    ("ai", "{answer}")
])

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt = example_prompt,
    examples = examples,
)

final_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a movie expert, you give  three emojis that represent the movie."),
    MessagesPlaceholder(variable_name="history"),
    example_prompt,
    ("human", "{question}")
])

"""
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful AI talking to a human"),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)
"""


def load_memory(_):
    return memory.load_memory_variables({})["history"]


chain = RunnablePassthrough.assign(history=load_memory) | final_prompt | llm


def invoke_chain(question):
    result = chain.invoke({"question": question})
    memory.save_context(
        {"input": question},
        {"output": result.content},
    )
    print(result)

In [2]:
invoke_chain("What do you know about Titanic?")

content='🚢❤️🌊'


In [3]:
invoke_chain("What do you know about Avatar?")

content='🪐🌿💙'


In [6]:
invoke_chain("What do you know about Iron Man 2?")

content='🦾💥🔴'


In [7]:
invoke_chain("tell me what is the movie I asked about first.")

content='The first movie you asked about was Titanic.'
